In [13]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import expr
spark = SparkSession.builder.appName("PracticeProject").enableHiveSupport().getOrCreate()
# Customers Data
customers_data = [
(101, 'Ali', 'ali@gmail.com', 'Mumbai', '2022-05-10'),
(102, 'Neha', 'neha@yahoo.com', 'Delhi', '2023-01-15'),
(103, 'Ravi', 'ravi@hotmail.com', 'Bangalore', '2021-11-01'),
(104, 'Sneha', 'sneha@outlook.com', 'Hyderabad', '2020-07-22'),
(105, 'Amit', 'amit@gmail.com', 'Chennai', '2023-03-10'),
]
orders_data = [
(1, 101, 'Laptop', 'Electronics', 2, 50000.0, '2024-01-10'),
(2, 101, 'Mouse', 'Electronics', 1, 1200.0, '2024-01-15'),
(3, 102, 'Tablet', 'Electronics', 1, 20000.0, '2024-02-01'),
(4, 103, 'Bookshelf', 'Furniture', 1, 3500.0, '2024-02-10'),
(5, 104, 'Mixer', 'Appliances', 1, 5000.0, '2024-02-15'),
(6, 105, 'Notebook', 'Stationery', 5, 500.0, '2024-03-01'),
(7, 102, 'Phone', 'Electronics', 1, 30000.0, '2024-03-02'),
]
customers_df = spark.createDataFrame(customers_data, ["CustomerID", "Name", "Email",
"City", "SignupDate"])
orders_df = spark.createDataFrame(orders_data, ["OrderID", "CustomerID", "Product",
"Category", "Quantity", "Price", "OrderDate"])

spark.sql("CREATE DATABASE IF NOT EXISTS sales")

customers_df.write.mode("overwrite").saveAsTable("sales.customers")
orders_df.write.mode("overwrite").saveAsTable("sales.orders")

In [15]:
from pyspark.sql.functions import col, lower, year, when

# 1. Add column TotalAmount = Price * Quantity
orders_df = orders_df.withColumn("TotalAmount", col("Price") * col("Quantity"))
orders_df.show()
# 2. Filter orders with TotalAmount > 10000
high_value_orders_df = orders_df.filter(col("TotalAmount") > 10000)
high_value_orders_df.show()

# 3. Standardize the City field in customers_df
customers_df = customers_df.withColumn("City", lower(col("City")))
customers_df.show()

# 4. Extract year from OrderDate and add OrderYear column
orders_df = orders_df.withColumn("OrderYear", year(col("OrderDate")))
orders_df.show()

# 5. Fill nulls in any column
customers_df = customers_df.fillna({'Email':'unknown@yahoo.com'})
customers_df.show()

# 6. Categorize orders based on TotalAmount
orders_df = orders_df.withColumn(
    "OrderCategory",
    when(col("TotalAmount") < 5000, "Low")
    .when((col("TotalAmount") >= 5000) & (col("TotalAmount") <= 20000), "Medium")
    .otherwise("High")
)
orders_df.show()



+-------+----------+---------+-----------+--------+-------+----------+-----------+---------+-------------+
|OrderID|CustomerID|  Product|   Category|Quantity|  Price| OrderDate|TotalAmount|OrderYear|OrderCategory|
+-------+----------+---------+-----------+--------+-------+----------+-----------+---------+-------------+
|      1|       101|   Laptop|Electronics|       2|50000.0|2024-01-10|   100000.0|     2024|         High|
|      2|       101|    Mouse|Electronics|       1| 1200.0|2024-01-15|     1200.0|     2024|          Low|
|      3|       102|   Tablet|Electronics|       1|20000.0|2024-02-01|    20000.0|     2024|       Medium|
|      4|       103|Bookshelf|  Furniture|       1| 3500.0|2024-02-10|     3500.0|     2024|          Low|
|      5|       104|    Mixer| Appliances|       1| 5000.0|2024-02-15|     5000.0|     2024|       Medium|
|      6|       105| Notebook| Stationery|       5|  500.0|2024-03-01|     2500.0|     2024|          Low|
|      7|       102|    Phone|Electro

In [16]:
# SECTION B: Spark SQL Tasks
# 7. Run a SQL query to list all orders made by “Ali”.
spark.sql("""
    select o.* from sales.orders o
    JOIN sales.customers c ON o.CustomerID = c.CustomerID
    where c.Name = 'Ali'
""").show()

# 8. Get total spending by each customer using SQL.
spark.sql("""
    select c.Name, SUM(o.Price * o.Quantity) AS TotalSpending
    from sales.orders o
    JOIN sales.customers c ON o.CustomerID = c.CustomerID
    GROUP BY c.Name
""").show()

# 9. Find out which category made the highest total revenue.
spark.sql("""
    select Category, SUM(Price * Quantity) AS Revenue
    from sales.orders
    GROUP BY Category
    ORDER BY Revenue DESC
    LIMIT 1
""").show()


+-------+----------+-------+-----------+--------+-------+----------+
|OrderID|CustomerID|Product|   Category|Quantity|  Price| OrderDate|
+-------+----------+-------+-----------+--------+-------+----------+
|      1|       101| Laptop|Electronics|       2|50000.0|2024-01-10|
|      2|       101|  Mouse|Electronics|       1| 1200.0|2024-01-15|
+-------+----------+-------+-----------+--------+-------+----------+

+-----+-------------+
| Name|TotalSpending|
+-----+-------------+
| Neha|      50000.0|
|  Ali|     101200.0|
| Ravi|       3500.0|
|Sneha|       5000.0|
| Amit|       2500.0|
+-----+-------------+

+-----------+--------+
|   Category| Revenue|
+-----------+--------+
|Electronics|151200.0|
+-----------+--------+



In [17]:
# 10. Create a view customer_orders showing CustomerName, Product, TotalAmount .

spark.sql("""
    CREATE OR REPLACE TEMP VIEW customer_orders AS
    select c.Name AS CustomerName, o.Product, (o.Price * o.Quantity) AS TotalAmount
    from sales.orders o
    JOIN sales.customers c ON o.CustomerID = c.CustomerID
""")

# 11. Query the view for products ordered after Feb 2024.
spark.sql("""
    select * from customer_orders
    JOIN sales.orders o ON customer_orders.Product = o.Product
    WHERE o.OrderDate > '2024-02-29'
""").show()


+------------+--------+-----------+-------+----------+--------+-----------+--------+-------+----------+
|CustomerName| Product|TotalAmount|OrderID|CustomerID| Product|   Category|Quantity|  Price| OrderDate|
+------------+--------+-----------+-------+----------+--------+-----------+--------+-------+----------+
|        Amit|Notebook|     2500.0|      6|       105|Notebook| Stationery|       5|  500.0|2024-03-01|
|        Neha|   Phone|    30000.0|      7|       102|   Phone|Electronics|       1|30000.0|2024-03-02|
+------------+--------+-----------+-------+----------+--------+-----------+--------+-------+----------+



In [18]:
# SECTION C: Advanced Practice
# 12. Create a Global Temp View from customers_df
customers_df.createOrReplaceGlobalTempView("customers")

spark.sql("SELECT * FROM global_temp.customers where City = 'mumbai'").show()

#13. Save the transformed orders_df (with TotalAmount) to a Parquet file.
orders_df.write.mode("overwrite").parquet("/tmp/orders_parquet")

#14. Read back the Parquet file and count how many orders are in it.
orders_parquet_df = spark.read.parquet("/tmp/orders_parquet")
orders_parquet_df.count()


+----------+----+-------------+------+----------+
|CustomerID|Name|        Email|  City|SignupDate|
+----------+----+-------------+------+----------+
|       101| Ali|ali@gmail.com|mumbai|2022-05-10|
+----------+----+-------------+------+----------+



7

In [19]:
# 15. Write a UDF that masks emails like: ali@gmail.com → a***@gmail.com .
from pyspark.sql.functions import udf
from pyspark.sql.types import StringType

def mask_email(email):
    try:
        name, domain = email.split('@')
        return name[0] + '***@' + domain
    except:
        return email

mask_email_udf = udf(mask_email, StringType())

customers_df = customers_df.withColumn("MaskedEmail", mask_email_udf(col("Email")))
customers_df.select("Email", "MaskedEmail").show()


+-----------------+----------------+
|            Email|     MaskedEmail|
+-----------------+----------------+
|    ali@gmail.com|  a***@gmail.com|
|   neha@yahoo.com|  n***@yahoo.com|
| ravi@hotmail.com|r***@hotmail.com|
|sneha@outlook.com|s***@outlook.com|
|   amit@gmail.com|  a***@gmail.com|
+-----------------+----------------+



In [20]:
# 16. Use concat_ws() to create a full label like: 'Ali from Mumbai'

from pyspark.sql.functions import concat_ws

customers_df = customers_df.withColumn("Label", concat_ws(" from ", col("Name"), col("City")))
customers_df.select("Name", "City", "Label").show()


+-----+---------+--------------------+
| Name|     City|               Label|
+-----+---------+--------------------+
|  Ali|   mumbai|     Ali from mumbai|
| Neha|    delhi|     Neha from delhi|
| Ravi|bangalore| Ravi from bangalore|
|Sneha|hyderabad|Sneha from hyderabad|
| Amit|  chennai|   Amit from chennai|
+-----+---------+--------------------+



In [21]:
#17. Use regexp_replace() to remove special characters from product names.
from pyspark.sql.functions import regexp_replace

orders_df = orders_df.withColumn("CleanProduct", regexp_replace(col("Product"), "[^a-zA-Z0-9 ]", ""))
orders_df.select("Product", "CleanProduct").show()


+---------+------------+
|  Product|CleanProduct|
+---------+------------+
|   Laptop|      Laptop|
|    Mouse|       Mouse|
|   Tablet|      Tablet|
|Bookshelf|   Bookshelf|
|    Mixer|       Mixer|
| Notebook|    Notebook|
|    Phone|       Phone|
+---------+------------+



In [22]:
# 18. Use to_date() and datediff() to calculate customer age in days (from
# SignupDate to today).
from pyspark.sql.functions import to_date, current_date, datediff

customers_df = customers_df.withColumn("SignupDate", to_date(col("SignupDate")))
customers_df = customers_df.withColumn("CustomerAgeDays", datediff(current_date(), col("SignupDate")))
customers_df.select("Name", "SignupDate", "CustomerAgeDays").show()


+-----+----------+---------------+
| Name|SignupDate|CustomerAgeDays|
+-----+----------+---------------+
|  Ali|2022-05-10|           1121|
| Neha|2023-01-15|            871|
| Ravi|2021-11-01|           1311|
|Sneha|2020-07-22|           1778|
| Amit|2023-03-10|            817|
+-----+----------+---------------+

